In [1]:
import pandas as pd
from matplotlib import pyplot as plt

plt.style.use('seaborn-whitegrid')

In [3]:
data = pd.read_csv('../data/principal_components.csv', parse_dates=['date']).set_index('date')

data.head()

,002А-Купе,005А-СВ,006А-СВ,701Н-Сидячий,703Н-Сидячий,704Н-Сидячий,705Н-Сидячий,706Н-Сидячий,707Н-Сидячий,708Н-Сидячий,...,777А-B2,777А-C1,777А-C2,778А-B2,778А-C2,779А-B2,779А-C1,779А-C2,780А-B2,780А-C2
date,,,,,,,,,,,,,,,,,,,,,
2021-04-12,0.980876,0.381817,1.793806,NaN,NaN,0.294729,NaN,0.615553,NaN,0.290559,...,0.801411,0.505144,0.386672,-0.156980,0.469554,1.394646,0.525794,0.745639,0.797818,0.933160
2021-04-13,0.970764,1.083278,1.470325,0.329414,0.903810,1.050396,NaN,0.732365,0.694859,0.639999,...,0.496001,0.887635,1.334755,-1.092759,-0.334608,1.642654,0.801214,1.138874,0.201627,0.677480
2021-04-14,0.719438,1.429311,0.535644,0.168052,0.397228,0.512671,1.142069,0.600286,0.559374,0.364613,...,1.047782,0.860062,1.462835,NaN,-1.244823,1.357480,0.923255,1.108289,-0.690236,0.387493
2021-04-15,0.400591,1.012460,0.422731,0.026877,0.380600,-0.059951,0.620381,0.101154,0.366308,-0.290051,...,1.015658,0.287210,1.332256,NaN,-1.271166,0.962127,0.887451,1.190646,NaN,-0.241193
2021-04-16,0.313064,1.274428,NaN,0.131996,-0.222185,-0.779462,-0.127400,-0.799788,-0.333775,NaN,...,-0.094477,0.508674,0.282571,NaN,-1.869963,1.116739,0.465230,0.536681,NaN,-1.542018


In [7]:
data.std(axis=0)

002А-Купе       0.880877
005А-СВ         1.191204
006А-СВ         1.247807
701Н-Сидячий    0.574946
703Н-Сидячий    0.601436
                  ...   
779А-B2         1.306067
779А-C1         1.060001
779А-C2         1.215287
780А-B2         0.824187
780А-C2         1.010888
Length: 67, dtype: float64

In [4]:
data = pd.read_csv('../data/train_clusters.csv', index_col='train')

data.head(2)

,mu,eig,both
train,,,
001А-Купе,1,3,1-3
002А-Купе,1,3,1-3


In [5]:
data['both'].value_counts()

1-3    20
0-0    17
2-1     7
0-3     7
1-2     6
0-1     2
2-0     2
1-0     1
Name: both, dtype: int64

In [2]:
data = pd.read_csv('../data/data.csv', parse_dates=['date'])

data['train'] = data['num'] + '-' + data['class']

data.head(2)

,date,days,places,price,num,class,weekday,day_name,weekend,train
0,2021-04-03,1,0.442857,3275.0,001А,Купе,5,Saturday,0,001А-Купе
1,2021-04-04,1,0.235714,3913.0,001А,Купе,6,Sunday,0,001А-Купе


In [3]:
grouped = data.groupby(['train', 'days'])['price'].mean().reset_index()\
    .pivot(index='train', columns='days', values='price')

grouped.head(2)

days,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30
train,,,,,,,,,,,,,,,,,,,,,
001А-Купе,3042.617647,3081.150000,2993.452381,2971.619048,2884.000000,2852.075,2797.595238,2766.238095,2746.833333,2682.976744,...,2783.969697,2804.516129,2815.633333,2840.642857,2840.250000,2799.777778,2847.769231,2860.130435,2879.166667,2898.952381
001А-СВ,6163.615385,6205.333333,5865.261905,5943.880952,5801.926829,5888.100,5940.738095,6058.690476,6097.833333,6088.651163,...,6183.151515,6212.322581,6189.733333,6112.821429,6177.964286,6149.370370,6120.884615,6070.260870,6056.791667,6043.190476


In [4]:
grouped.shape

(144, 30)

In [6]:
trains = grouped.index.tolist()

ncols = 6
nrows = len(trains) // ncols + (len(trains) % ncols > 0)

fig, axes = plt.subplots(ncols=ncols, nrows=nrows)
fig.set_size_inches(3 * ncols, 3 * nrows)
fig.subplots_adjust(wspace=0.5, hspace=0.6)
axes = axes.flatten()

for train, ax in zip(trains, axes):
    row = grouped.loc[train]
    ax.plot(row, c='C3', lw=2)
    ax.fill_between(row.index, row.values, row.min(), color='C3', alpha=0.15)
    
    ax.set_title(train)
    ax.set_xlabel('days to departure')
    ax.set_ylabel('avg. ticket price')
    ax.invert_xaxis()
    
for i in range(len(trains), len(axes)):
    axes[i].axis('off')
    
plt.savefig('../data/figures/price_vs_days.pdf', transparent=True, bbox_inches='tight')
plt.close()

In [10]:
grouped = data.groupby(['train', 'days'])['places'].mean().reset_index()\
    .pivot(index='train', columns='days', values='places')

grouped.head(2)

days,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30
train,,,,,,,,,,,,,,,,,,,,,
001А-Купе,0.382784,0.436905,0.498810,0.549830,0.613415,0.636786,0.675510,0.707143,0.726701,0.751163,...,0.882900,0.885023,0.890238,0.902296,0.897194,0.897090,0.904121,0.897205,0.902381,0.907483
001А-СВ,0.605569,0.627604,0.686756,0.704241,0.734566,0.760352,0.764137,0.781436,0.794457,0.799964,...,0.778172,0.782258,0.795052,0.798549,0.796596,0.813079,0.808293,0.829144,0.838867,0.873512


In [11]:
grouped.shape

(144, 30)

In [14]:
trains = grouped.index.tolist()

ncols = 6
nrows = len(trains) // ncols + (len(trains) % ncols > 0)

fig, axes = plt.subplots(ncols=ncols, nrows=nrows)
fig.set_size_inches(3 * ncols, 3 * nrows)
fig.subplots_adjust(wspace=0.5, hspace=0.6)
axes = axes.flatten()

for train, ax in zip(trains, axes):
    row = grouped.loc[train]
    ax.plot(row, c='C2', lw=2)
    ax.fill_between(row.index, row.values, row.min(), color='C2', alpha=0.15)
    
    ax.set_title(train)
    ax.set_xlabel('days to departure')
    ax.set_ylabel('avg. vacant places')
    ax.invert_xaxis()
    
for i in range(len(trains), len(axes)):
    axes[i].axis('off')
    
plt.savefig('../data/figures/places_vs_days.pdf', transparent=True, bbox_inches='tight')
plt.close()